In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls

In [2]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/proportion_pxt_neh/"
time=31
pw=0.046

In [3]:
data=pd.read_csv(path+"__CPSM_MOUSE.csv")
data=data[data.exp_Time==time]
data=data[data.exp_Time==time].reset_index(drop="True")
data.head()

,Peptied,charge,exp_Time,NEH,e_NEH,pxt,e_pxt,mo,m1,m2,m3,m4,m5,io,i1,i2,i3,i4,i5
0,SMEYcIIEVNAR,2,31,25,22,0.046,0.050,0.40995,0.31550,0.17244,0.07014,0.02338,0.00661,0.131710,0.261299,0.278459,0.183605,0.105282,0.039646
1,IEFEGQSVDFVDPNK,2,31,28,27,0.046,0.050,0.36570,0.34991,0.18540,0.07061,0.02144,0.00548,0.099898,0.215021,0.271999,0.214834,0.134417,0.063831
2,TAVDSGIALLTNFQVTK,2,31,25,26,0.046,0.045,0.35488,0.35097,0.19026,0.07374,0.02269,0.00586,0.112706,0.241696,0.267777,0.206311,0.116566,0.054945
3,EPLFGISTGNIITGLAAGAK,3,31,36,32,0.046,0.048,0.32091,0.34919,0.20558,0.08588,0.02834,0.00782,0.075000,0.185255,0.252018,0.237232,0.162041,0.088454
4,PSYVLSGSAMNVVFSEDEMK,2,31,36,32,0.046,0.048,0.26161,0.31262,0.22573,0.12075,0.05225,0.01916,0.055720,0.167710,0.233795,0.236165,0.186275,0.120335


In [4]:
res=[]
for pep_index in range(data.shape[0]):
    exp_0=list(data.loc[pep_index,["mo","m1","m2","m3","m4","m5"]])
    b=list(data.loc[pep_index,["io","i1","i2","i3","i4","i5"]])#exp_t
    
    A=[]
    for i in range(6):
        A.append(([0]*i)+exp_0[:6-i])
    A=np.array(A).transpose()
#     x=np.matmul( np.linalg.inv ( np.matmul(A.transpose(),A)) , np.matmul(A.transpose(),b)  )
    x=list(nnls(A,b)[0])
    
    neh= (x[3]/x[2])*3* ((1-pw)/pw) +2
    xm=sum([(index+1)*val / sum(x) for index,val in enumerate(x)])
    res.append([data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],data.loc[pep_index,"NEH"],neh,xm, 
               data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"]])
    
    ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
    sigma=sum([ (b-x)**2 for x in ypred])/(len(ypred)-6-1)
    cov=((sigma**2)*np.linalg.inv(np.matmul(A.transpose(),A)))
    var=[cov[x][x] for x in range(6)]
    
#     print(data.loc[pep_index,"Peptied"],"\n","\n",var,"\n")

#     print(data.loc[pep_index,"Peptied"],x,end="\n\n")

In [5]:
res=pd.DataFrame(res)
res.columns=["Peptide","Charge","True NEH","e NEH","sum_xi","pxt_neh"]
res["dif"]=abs(res["True NEH"]- res["e NEH"])
res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]
res=res.sort_values(by="Peptide")
res

,Peptide,Charge,True NEH,e NEH,sum_xi,pxt_neh,dif,test
41,AADTIGYPVMIR,2,22,22.506434,1.909365,0.920000,0.506434,107.539740
6,AFAISGPFNVQFLVK,2,25,25.990336,2.073809,1.049999,0.990336,97.505711
46,AFAmTNQILVER,2,25,28.224703,2.141740,1.056000,3.224703,102.816299
45,AMLSTGFK,2,11,2.000000,1.400172,0.423000,9.000000,231.010075
15,AVNTLNEALEFANSVGYPcLLR,3,42,44.332053,2.810590,1.749999,2.332053,60.605228
36,DGSIDLVINLPNNNTK,3,23,22.855625,1.947530,0.930999,0.144375,109.187041
32,DGSIDLVINLPNNNTK,2,23,25.102248,2.131078,1.022000,2.102248,108.520311
47,EIEYEVVR,2,17,26.328325,1.861650,0.856000,9.328325,117.482480
11,EPLFGISTGNIITGL,2,22,19.938711,2.063372,0.949000,2.061289,117.425858
3,EPLFGISTGNIITGLAAGAK,3,36,37.161638,2.533843,1.535999,1.161638,64.963806


In [6]:
ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
sigma=sum([ (b-x)**2 for x in ypred])/(len(ypred)-6-1)
cov=((sigma**2)*np.linalg.inv(np.matmul(A.transpose(),A)))
var=[cov[x][x] for x in range(6)]

ci_ub=[x[i]+1.96*var[i] for i in range(6)]
ci_lb=[x[i]-1.96*var[i] for i in range(6)]
ci=[1.96*var[i] for i in range(6)]

pd.DataFrame([x,ci,ci_ub,ci_lb]).transpose()

# temp=pd.DataFrame()
# temp["AA"]=all_AAs1
# temp["E_value"]=sol
# temp["T_value"]=trueval
# temp["var"]=var
# temp["CI"]=ci
# temp.sort_values(by="var")

,0,1,2,3
0,0.384758,0.046401,0.431160,0.338357
1,0.396391,0.433260,0.829650,-0.036869
2,0.168970,0.104200,0.273170,0.064771
3,0.041565,0.051272,0.092838,-0.009707
4,0.007174,0.183117,0.190291,-0.175943
5,0.011726,0.305973,0.317699,-0.294247


In [7]:
res.to_csv("testx3x2.csv",index=False)